In [1]:
import pickle

In [19]:
import pythonwhois  # i'm using this http://cryto.net/pythonwhois

In [28]:
from urllib.error import HTTPError
from requests.exceptions import HTTPError


In [29]:
import time
import random


In [34]:
import urllib

In [30]:
## function that forces system to wait random time, with upper bound on wait equals to maxTime
def waitRandomTime(maxTime=7):
    ## setting up the seed
    tm = int(round(time.time()))
    random.seed(tm)

    tim = random.randint(0, 8 * 100) / 100.0
    time.sleep(tim)



In [209]:
### safely open the saved pickle file
def safelyOpen(nameOfSavedDictionary):
    try:
        with open(nameOfSavedDictionary + '.pickle', 'rb') as handle:
            return pickle.load(handle)
    except (NameError, FileNotFoundError):
        print("File was not found ")

In [20]:
def uniqify(seq):
    # Not order preserving
    keys = {}
    for e in seq:
        keys[e] = 1
    return list(keys.keys())

In [12]:
dataFake = safelyOpen('fakeKaggleEnglishOver300')

In [13]:
sites = dataFake.ix[:,'site_url']

In [26]:
sites = uniqify(sites)

In [166]:
domains = ['nytimes.com']
for dom in domains:
    details = pythonwhois.get_whois(dom)
    print(details)  

{'id': ['91991_DOMAIN_COM-VRSN'], 'status': ['clientUpdateProhibited (https://www.icann.org/epp#clientUpdateProhibited)', 'clientTransferProhibited (https://www.icann.org/epp#clientTransferProhibited)', 'clientDeleteProhibited (https://www.icann.org/epp#clientDeleteProhibited)', 'serverUpdateProhibited (https://www.icann.org/epp#serverUpdateProhibited)', 'serverTransferProhibited (https://www.icann.org/epp#serverTransferProhibited)', 'serverDeleteProhibited (https://www.icann.org/epp#serverDeleteProhibited)'], 'creation_date': [datetime.datetime(1994, 1, 17, 21, 0)], 'expiration_date': [datetime.datetime(2023, 1, 18, 21, 0), datetime.datetime(2023, 1, 18, 21, 0)], 'updated_date': [datetime.datetime(2016, 10, 21, 15, 12, 35)], 'registrar': ['MarkMonitor, Inc.'], 'whois_server': ['whois.markmonitor.com'], 'nameservers': ['dns-plx.ewr1.nytimes.com', 'ns2.p24.dynect.net', 'dns-plx.sea1.nytimes.com', 'ns4.p24.dynect.net', 'ns3.p24.dynect.net', 'ns1.p24.dynect.net'], 'emails': ['abusecomplai

In [168]:
details.keys()

dict_keys(['id', 'status', 'creation_date', 'expiration_date', 'updated_date', 'registrar', 'whois_server', 'nameservers', 'emails', 'contacts', 'raw'])

In [172]:
details['creation_date']

[datetime.datetime(1994, 1, 17, 21, 0)]

In [192]:
def find_url_meta(domains, meta_fake):
    if type(meta_fake) == dict:
        for dom in domains:
            try:
                print(dom)
                meta_fake[dom] = pythonwhois.get_whois(dom)
            except (urllib.request.HTTPError, HTTPError, ConnectionResetError, UnicodeDecodeError):
                waitRandomTime(45)
            except KeyError:
                meta_fake[dom] = None
            except pythonwhois.shared.WhoisException:
                meta_fake[dom] = None
    else:
        raise ValueError
        
        

In [81]:
sites[57]

'endtime.com'

In [84]:
meta_fake_old = meta_fake

In [102]:
meta_fake = {}

In [105]:
find_url_meta(sites, meta_fake)

100percentfedup.com
21stcenturywire.com
abcnews.com.co
abeldanger.net
abovetopsecret.com
activistpost.com
addictinginfo.org
adobochronicles.com
ahtribune.com
allnewspipeline.com
americanlookout.com
americasfreedomfighters.com
amren.com
amtvmedia.com
anonews.co
anonhq.com
antiwar.com
awdnews.com
barenakedislam.com
beforeitsnews.com
betootaadvocate.com
bigbluevision.org
bignuggetnews.com
bipartisanreport.com
blackagendareport.com
blacklistednews.com
breitbart.com
christiantimesnewspaper.com
chronicle.su
clickhole.com
coasttocoastam.com
collective-evolution.com
consciouslifenews.com
conservativedailypost.com
conservativetribune.com
consortiumnews.com
corbettreport.com
countercurrents.org
counterpunch.org
dailynewsbin.com
dailysquib.co.uk
dailystormer.com
dailywire.com
darkmoon.me
davidduke.com
davidstockmanscontracorner.com
davidwolfe.com
dcclothesline.com
defenddemocracy.press
dennismichaellynch.com
departed.co
disclose.tv
donaldtrumpnews.co
educateinspirechange.org
empireherald.com
endi

 sites

In [108]:
len(meta_fake.keys())

220

In [112]:
for key in meta_fake.keys():
    if meta_fake[key] == None:
        print(key)

globalresearch.ca
theearthchild.co.za


In [152]:
meta_fake['100percentfedup.com']

{'contacts': {'admin': None,
  'billing': None,
  'registrant': None,
  'tech': None},
 'creation_date': [datetime.datetime(2012, 3, 13, 0, 0)],
 'expiration_date': [datetime.datetime(2020, 9, 29, 0, 0)],
 'nameservers': ['ED.NS.CLOUDFLARE.COM', 'GINA.NS.CLOUDFLARE.COM'],
 'raw': ['Domain Name: 100PERCENTFEDUP.COM\nRegistrar URL: http://www.godaddy.com\nRegistrant Name: Registration Private\nRegistrant Organization: Domains By Proxy, LLC\nName Server: ED.NS.CLOUDFLARE.COM\nName Server: GINA.NS.CLOUDFLARE.COM\nDNSSEC: unsigned\n\nFor complete domain details go to:\nhttp://who.godaddy.com/whoischeck.aspx?domain=100PERCENTFEDUP.COM\n\nThe data contained in GoDaddy.com, LLC\'s WhoIs database,\nwhile believed by the company to be reliable, is provided "as is"\nwith no guarantee or warranties regarding its accuracy.  This\ninformation is provided for the sole purpose of assisting you\nin obtaining information about domain name registration records.\nAny use of this data for any other purpose

In [179]:
for xx in meta_fake.keys():
    try:
        print(meta_fake[xx]['creation_date'])
    except (TypeError,KeyError):
        print('no creation date for ' + xx)
    

[datetime.datetime(2012, 3, 13, 0, 0)]
[datetime.datetime(2009, 11, 3, 0, 0)]
[datetime.datetime(2015, 10, 26, 21, 56, 34), datetime.datetime(2015, 10, 26, 21, 56, 34)]
[datetime.datetime(2010, 5, 12, 0, 0)]
[datetime.datetime(1997, 5, 18, 0, 0)]
[datetime.datetime(2010, 4, 23, 0, 0)]
[datetime.datetime(2010, 9, 5, 20, 33, 25)]
[datetime.datetime(2013, 6, 25, 0, 0)]
[datetime.datetime(2015, 8, 23, 8, 8, 39)]
[datetime.datetime(2014, 7, 7, 0, 0)]
[datetime.datetime(2016, 5, 10, 0, 0)]
[datetime.datetime(2013, 12, 17, 10, 41, 40)]
[datetime.datetime(1995, 8, 11, 4, 0)]
[datetime.datetime(2012, 1, 31, 0, 0)]
[datetime.datetime(2015, 12, 16, 22, 42, 2), datetime.datetime(2015, 12, 16, 22, 42, 2)]
[datetime.datetime(2014, 4, 11, 0, 0)]
[datetime.datetime(1995, 12, 9, 0, 0)]
[datetime.datetime(2012, 1, 16, 4, 0)]
[datetime.datetime(2008, 7, 9, 6, 5, 5)]
[datetime.datetime(2007, 9, 15, 0, 0)]
[datetime.datetime(2014, 10, 2, 7, 17, 29)]
[datetime.datetime(2016, 7, 25, 11, 4, 37)]
[datetime.dat

In [180]:
## query for the true set

In [189]:
good_urls = ['nytimes.com', 'wsj.com', 'abcnews.go.com', 'cnn.com', 'cbsnews.com', 'foxnews.com', 'msnbc.com',
                  'nbcnews.com','oann.com', 'latimes.com', 'usatoday.com', 'washingtonpost.com', 'newsweek.com',
                  'time.com', 'usnews.com', 'theguardian.com', 'telegraph.co.uk', 'thetimes.co.uk', 'ft.com',
                  'independent.co.uk', 'bbc.com', 'standard.co.uk', 'dailymail.co.uk', 'express.co.uk',
                  'dailytelegraph.com.au', 'thestar.com','theglobeandmail.com', 'nationalpost.com',
                   'calgaryherald.com', 'herald.ie', 'irishtimes.com', 'independent.ie', 'afr.com.au',
                   'theaustralian.com.au', 'thesaturdaypaper.com.au', 'reddit.com', 'Cnn.com', 'Bbc.co.uk', 'Weather.com',
                   'News.yahoo.com', 'Huffingtonpost.com','Forbes.com', 'Foxnews.com', 'news.google.com' ,
                   'Shutterstock.com', 'Timesofindia.indiatimes.com', 'Bloomberg.com', 'Reuters.com', 'Wunderground.com',
                   'Money.cnn.com', 'Indianexpress.com', 'Nbcnews.com', 'Latimes.com', 'cnbc.com', 'cbsnews.com',
                   'vox.com', 'Abcnews.go.com', 'Nypost.com', 'Theatlantic.com', 'Chicagotribune.com', 'Chinadaily.com.cn'
                   ,'Hollywoodreporter.com', 'Sfgate.com', 'Usnews.com', 'Economist.com', 'Aljazeera.com', 'Fortune.com',
                   'Newsnow.co.uk', 'Variety.com', 'Euronews.com', 'Washingtontimes.com', 'Bostonglobe.com', 'Newsweek.com'
                   ]

In [190]:
meta_true = {}

In [193]:
find_url_meta(good_urls, meta_true)

nytimes.com
wsj.com
abcnews.go.com
cnn.com
cbsnews.com
foxnews.com
msnbc.com
nbcnews.com
oann.com
latimes.com
usatoday.com
washingtonpost.com
newsweek.com
time.com
usnews.com
theguardian.com
telegraph.co.uk
thetimes.co.uk
ft.com
independent.co.uk
bbc.com
standard.co.uk
dailymail.co.uk
express.co.uk
dailytelegraph.com.au
thestar.com
theglobeandmail.com
nationalpost.com
calgaryherald.com
herald.ie
irishtimes.com
independent.ie
afr.com.au
theaustralian.com.au
thesaturdaypaper.com.au
reddit.com
Cnn.com
Bbc.co.uk
Weather.com
News.yahoo.com
Huffingtonpost.com
Forbes.com
Foxnews.com
news.google.com
Shutterstock.com
Timesofindia.indiatimes.com
Bloomberg.com
Reuters.com
Wunderground.com
Money.cnn.com
Indianexpress.com
Nbcnews.com
Latimes.com
cnbc.com
cbsnews.com
vox.com
Abcnews.go.com
Nypost.com
Theatlantic.com
Chicagotribune.com
Chinadaily.com.cn
Hollywoodreporter.com
Sfgate.com
Usnews.com
Economist.com
Aljazeera.com
Fortune.com
Newsnow.co.uk
Variety.com
Euronews.com
Washingtontimes.com
Boston

In [194]:
for xx in meta_true.keys():
    try:
        print(xx)
        print(meta_true[xx]['creation_date'])
    except (TypeError,KeyError):
        print('no creation date for ' + xx)
    

nytimes.com
[datetime.datetime(1994, 1, 17, 21, 0)]
wsj.com
[datetime.datetime(1994, 5, 26, 4, 0)]
abcnews.go.com
no creation date for abcnews.go.com
cnn.com
[datetime.datetime(1993, 9, 22, 4, 0)]
cbsnews.com
[datetime.datetime(1994, 10, 4, 0, 0)]
foxnews.com
[datetime.datetime(1995, 6, 21, 0, 0)]
msnbc.com
[datetime.datetime(1995, 12, 14, 21, 0)]
nbcnews.com
[datetime.datetime(1997, 6, 25, 21, 0)]
oann.com
[datetime.datetime(2004, 8, 12, 18, 41, 11)]
latimes.com
[datetime.datetime(1990, 12, 11, 21, 0)]
usatoday.com
[datetime.datetime(1994, 4, 29, 0, 0)]
washingtonpost.com
[datetime.datetime(1995, 11, 13, 5, 0)]
newsweek.com
[datetime.datetime(1994, 5, 16, 21, 0)]
time.com
[datetime.datetime(1993, 11, 19, 0, 0)]
usnews.com
[datetime.datetime(1995, 1, 20, 5, 0)]
theguardian.com
[datetime.datetime(1994, 7, 6, 0, 0)]
telegraph.co.uk
no creation date for telegraph.co.uk
thetimes.co.uk
[datetime.datetime(1996, 8, 23, 0, 0), datetime.datetime(1996, 8, 23, 0, 0), datetime.datetime(1996, 8, 23

In [198]:
import datetime

In [199]:
## ok lets create outside nlp feature

In [200]:
def year_bias(year):
    if year<2000:
        return 1
    elif year>2000:
        return -1
    else:
        return 0

In [213]:
fla = safelyOpen('chromda')

File was not found 


In [215]:
fla is None


True